In [4]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [5]:
lines= pd.read_table('uniquew.txt', names=['eng', 'tel'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [6]:
lines.shape

(2232, 2)

In [7]:
lines.eng=lines.eng.str.lower()#apply(lambda x: x.lower())
lines.tel=lines.tel.str.lower()

In [8]:
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', str(x)))
lines.tel=lines.tel.apply(lambda x: re.sub("'", '', str(x)))

In [9]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.tel=lines.tel.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [10]:
lines.eng

0              ­ఊహించలేరు
1                   అంచనా
2                  అంటాడు
3                  అంటారు
4                 అంటున్న
5                    అంటూ
6                    అంటే
7                  అంటేనే
8                 అంటోంది
9                   అండగా
10                  అండ్‌
11                  అంతకు
12                  అంతగా
13                అంతర్గత
14             అంతర్గతంగా
15                   అంతా
16               అంతులేని
17                   అంతే
18                 అందమైన
19                అందరికీ
20                  అందరూ
21                  అందాక
22                 అందాలు
23                   అంది
24                అందినంత
25              అందుకుంది
26           అకస్మాత్తుగా
27                  అక్కడ
28               అక్కడికీ
29                 అక్కడే
              ...        
2202             స్థాయిని
2203             స్థితిలో
2204               స్నేహం
2205           స్నేహితులు
2206             స్పందనకు
2207        స్పర్శనిబట్టి
2208                స్పృహ
2209        

In [11]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.tel = lines.tel.apply(lambda x: re.sub("[0123456789]", "", x))

In [12]:
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.tel=lines.tel.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.tel=lines.tel.apply(lambda x: re.sub(" +", " ", x))

lines.tel = lines.tel.apply(lambda x : 'START_ '+ x + ' _END')

In [13]:
lines.sample(10)

,eng,tel
672,కౌన్సిలింగ్‌,START_ కౌన్సిల్‌ _END
262,ఇటు,START_ ఇటు _END
1455,పోనిచ్చాడు,START_ పోయి ఇచ్చు _END
937,జరిపి,START_ జరగు _END
907,చేస్తారు,START_ చేయు _END
165,అవలంబించాలి,START_ అవలంబించు _END
854,చెల్లెలికన్నా,START_ చెల్లెలు _END
401,ఎదురయ్యే,START_ ఎదురు అవ్వు _END
1459,పోలికలను,START_ పోలిక _END
1449,పొరపాట్లు,START_ పొరపాటు _END


In [14]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [15]:
all_Telugu_words=set()
for tel in lines.tel:
    for word in tel.split():
        if word not in all_Telugu_words:
            all_Telugu_words.add(word)

In [16]:
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print(max_length_src)

2


In [17]:
lenght_list=[]
for l in lines.tel:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)

print(max_length_tar)

5


In [18]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_Telugu_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_Telugu_words)
print(num_encoder_tokens, num_decoder_tokens)
num_decoder_tokens += 1

2210 1215


In [19]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [20]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [21]:
lines = shuffle(lines)
print(lines.head(10))

              eng                       tel
1417   పెట్టుకునే    START_ పెట్టు అను _END
1148     దిగులుగా        START_ దిగులు _END
1624       మధ్యగల       START_ మధ్య కల _END
574         కావడం         START_ కావడం _END
634        కొంచెం        START_ కొంచెం _END
1733       మునిగి        START_ మునుగు _END
1862         లేరు          START_ లేరు _END
1666    మలచుకోవడం          START_ మలచు _END
2088  శనాదివారాలు  START_ శని ఆది వారం _END
1691      మానిక్‌       START_ మానిక్‌ _END


In [22]:
X, y = lines.eng, lines.tel
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
print(X_train.shape, X_test.shape)

(2008,) (224,)


In [23]:
X_train.to_pickle('Weights_Tel/X_train.pkl')
X_test.to_pickle('Weights_Tel/X_test.pkl')

In [24]:
def generate_batch(X = X_train, y = y_train, batch_size = 20):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src+1),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar+1),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar+1, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [25]:
latent_dim = 50

In [26]:

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [27]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [28]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [29]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 20
epochs = 20
val_samples
train_samples

2008

In [30]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = int(train_samples/batch_size),
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = int(val_samples/batch_size))


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/20
100/100 [==============================] - 15s 152ms/step - loss: 5.3022 - acc: 0.4543 - val_loss: 3.9337 - val_acc: 0.4512
Epoch 2/20
100/100 [==============================] - 8s 84ms/step - loss: 3.8246 - acc: 0.4602 - val_loss: 3.9403 - val_acc: 0.4521
Epoch 3/20
100/100 [==============================] - 9s 85ms/step - loss: 3.7734 - acc: 0.4600 - val_loss: 3.9897 - val_acc: 0.4484
Epoch 4/20
100/100 [==============================] - 9s 86ms/step - loss: 3.7002 - acc: 0.4601 - val_loss: 4.0459 - val_acc: 0.4503
Epoch 5/20
100/100 [==============================] - 9s 85ms/step - loss: 3.6250 - acc: 0.4598 - val_loss: 4.0968 - val_acc: 0.4503
Epoch 6/20
100/100 [==============================] - 9s 85ms/step - loss: 3.5680 - acc: 0.4601 - val_loss: 4.1458 - val_acc: 0.4512
Epoch 7/20
100/100 [==============================] - 8s 84ms/step - loss:

In [31]:
model.save_weights('nmt_weights.h5')

In [32]:
model.load_weights('nmt_weights.h5')

In [33]:
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [34]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [35]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [59]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Tel Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Tel Translation:', decoded_sentence[:-4])

Input English sentence: నేను
Actual Tel Translation:  నేను 
Predicted Tel Translation:  మరి 
